# Introduction
1. This system that relies on semantic similarity. It finds the text in the document that is most similar to the user's question.
2. If the user's question doesn't closely resemble the way the information is expressed in the document, the system may not find the correct answer.
3. Basic Functionality covers:
    * Extract text from PDF documents.
    * Perform semantic search to find relevant chunks of text.
    * Clean the output to remove unwanted content.
    * Provide an answer to the user's question (even if the answer is not always perfect).



## Further Development
1. Clarifying Expectation, example :
    * Chatbot: "Dana BOS digunakan untuk membiayai kegiatan operasional sekolah. Apakah Anda ingin mengetahui contoh kegiatan operasional yang dapat dibiayai oleh Dana BOS?"
2. Provide a list of example questions that the user can ask. This shows them the types of questions the chatbot is good at answering. Example:
    * Apa saja syarat pengajuan Dana BOS?
    * Bagaimana cara melaporkan penggunaan Dana BOS?
    * Sebutkan contoh kegiatan yang dapat dibiayai oleh Dana BOS.
3. Keyword Suggestions: As the user types their question, suggest relevant keywords that they can include to make their question more specific.
4. Intent Recognition (Advanced): Implement a simple intent recognition system. This would analyze the user's question and try to identify the intent behind it (e.g., "find allowed uses," "find reporting requirements"). Based on the intent, the chatbot could automatically rephrase the question to be more targeted. This requires more advanced natural language processing techniques.
5. Expand the Training Data (If Possible): If you have the ability to add more data to the system, try to find documents that explicitly list the allowed uses of Dana BOS in a clear and structured way. This will make it easier for the semantic search to find the right information.
6. Hybrid Approach (Advanced): Consider combining this semantic search approach with a more traditional keyword-based search. If the semantic search fails to find a good answer, the chatbot could fall back to a keyword search to find any relevant documents and present them to the user.

# Import Library

In [113]:
!pip install pymupdf nltk transformers sentence-transformers faiss-cpu

import os
import re
import json
import fitz
import nltk
import faiss
import numpy as np

from nltk import sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer, CrossEncoder
from google.colab import drive

# Download resource NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data Gathering

In [114]:
# Data Gathering
# ===============================
# 1. MOUNT GOOGLE DRIVE & CEK FILES
# ===============================

# Mount Google Drive
drive.mount('/content/drive')

# Path ke direktori penyimpanan file PDF
pdf_dir = "/content/drive/My Drive/Colab Notebooks/AI Chatbot Berbasis Regulasi"

# Cek apakah direktori ada
if not os.path.exists(pdf_dir):
    raise FileNotFoundError(f"Direktori {pdf_dir} tidak ditemukan! Periksa kembali path-nya.")
else:
    print(f"Direktori ditemukan! Daftar file PDF: {os.listdir(pdf_dir)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Direktori ditemukan! Daftar file PDF: ['Permendikbudriset No. 63 Tahun 2023.pdf', 'Untitled folder', 'embeddings.npy', 'chunks.json', 'cleaned_texts.json']


In [115]:
# ===============================
# 2. EKSTRAKSI TEKS DARI FILE PDF
# ===============================

# --- PDF Text Extraction ---
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]
pdf_texts = {}

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    try:
        text = extract_text_from_pdf(pdf_path)
        pdf_texts[pdf_file] = text
        print(f"Extracted text from: {pdf_file}")
    except Exception as e:
        print(f"Error extracting text from {pdf_file}: {e}")

Extracted text from: Permendikbudriset No. 63 Tahun 2023.pdf


# Preprocessing Data

In [116]:
# ===============================
# 3. PREPROCESSING TEKS
# ===============================

def clean_text(text):

    # This collapses multiple consecutive blank lines into a single blank line,
    # reducing unnecessary whitespace.
    text = re.sub(r'\n+', '\n', text)

    # replaces sequences of spaces, tabs, or newlines with a single space,
    # ensuring consistent spacing
    text = re.sub(r'\s+', ' ', text).strip()

    # This line finds instances like "Pasal 17." and replaces them with
    # "Pasal 17 ". It removes the dot after the number and ensures
    # there is space. This prevents the sentence tokenizer from incorrectly
    # splitting "Pasal 17." into two sentences. It's important to keep
    # "Pasal 17" together as a single unit.
    text = re.sub(r'Pasal (\d+)\.\s', r'Pasal \1 ', text)

    # Remove dot, KEEP contents of parentheses
    text = re.sub(r'Ayat \((\d+[a-z]?)\)\.\s', r'Ayat (\1) ', text)

    text = re.sub(r'http\S+|www\S+', '', text, flags=re.IGNORECASE)  # Remove URLs
    text = re.sub(r'jdih\.kemdikbud\.go\.id', '', text, flags=re.IGNORECASE)  # Remove specific website

    # Replace page number pattern '- 4 -' with '(page 4)'
    text = re.sub(r'\s-\s(\d+)\s-\s', r' (page \1) ', text)

    return text

cleaned_texts = {pdf: clean_text(text) for pdf, text in pdf_texts.items()}

In [117]:
# ===============================
# 4. CHUNKING TEKS
# Splits text into smaller chunks.
# ===============================

def chunk_text(text, chunk_size=500):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) + 1 <= chunk_size:
            current_chunk += sentence + " "
        else:
            if len(current_chunk) > 100:  # Pastikan chunk tidak terlalu kecil
                chunks.append(current_chunk.strip())
            current_chunk = sentence + " "

    if len(current_chunk) > 100:  # Pastikan chunk tidak terlalu kecil
        chunks.append(current_chunk.strip())

    return chunks

def clean_chunk(chunk):
    # Hapus angka atau simbol yang berada di awal baris
    chunk = re.sub(r'^\s*[\d\-\•]+', '', chunk)
    # Hapus angka yang berdiri sendiri tanpa konteks
    chunk = re.sub(r'\s*\d+\s*$', '', chunk)
    return chunk.strip()

def filter_irrelevant_text(chunk):
    irrelevant_patterns = [
        r'\(\d+\)\s*Dihapus',  # "(3) Dihapus", "(4) Dihapus"
        r'-\d+-',  # "-9-", "-11-" (kemungkinan nomor halaman)
        r'Pasal\s*\d+',  # "Pasal 52a" (jika tidak ada konteks)
        r'^\s*\.\s*$',  # Tanda titik yang berdiri sendiri
    ]

    for pattern in irrelevant_patterns:
        chunk = re.sub(pattern, '', chunk)

    # Hapus angka yang berdiri sendiri, kecuali yang ada dalam kurung ( ) atau { }
    chunk = re.sub(r'\b\d+\b(?![\)}])', ' ', chunk)  # Hanya hapus angka yang tidak diikuti kurung

    # Hilangkan spasi berlebihan
    chunk = re.sub(r'\s+', ' ', chunk).strip()

    return chunk

all_chunks = []
for pdf, text in cleaned_texts.items():
    chunks = chunk_text(text)  # 1️⃣ Chunking dulu
    cleaned_chunks = [clean_chunk(chunk) for chunk in chunks]  # 2️⃣ Clean angka awal
    final_chunks = [filter_irrelevant_text(chunk) for chunk in cleaned_chunks]  # 3️⃣ Hapus bagian tidak relevan
    all_chunks.extend(final_chunks)

print(f"Total chunks: {len(all_chunks)}")

Total chunks: 66


# SAVING DATA

In [118]:
# ===============================
# 5. SAVING DATA
# ===============================

# Define file paths for saving data
chunks_file = os.path.join(pdf_dir, "chunks.json")  # Path to save chunks
cleaned_texts_file = os.path.join(pdf_dir, "cleaned_texts.json") # Path to save cleaned texts

# --------------------------------------
# 1. Saving the Chunks of Text
# --------------------------------------
try:
    with open(chunks_file, "w", encoding="utf-8") as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=4)
    print(f"Chunks saved to: {chunks_file}")
except Exception as e:
    print(f"Error saving chunks: {e}")

# --------------------------------------
# 2. Saving the Cleaned PDF Texts
# --------------------------------------
try:
    with open(cleaned_texts_file, "w", encoding="utf-8") as f:
        json.dump(cleaned_texts, f, ensure_ascii=False, indent=4)
    print(f"Cleaned texts saved to: {cleaned_texts_file}")
except Exception as e:
    print(f"Error saving cleaned texts: {e}")

Chunks saved to: /content/drive/My Drive/Colab Notebooks/AI Chatbot Berbasis Regulasi/chunks.json
Cleaned texts saved to: /content/drive/My Drive/Colab Notebooks/AI Chatbot Berbasis Regulasi/cleaned_texts.json


# LOAD MODEL

In [119]:
# ===============================
# 6. MODEL
# ===============================

# Load Sentence Transformer model
cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# ===============================
# 7. FAISS INDEXING
# ===============================

d = 384  # Dimensi embedding dari model MiniLM
index = faiss.IndexFlatL2(d)
chunk_embeddings = embedder.encode(all_chunks, convert_to_numpy=True)
index.add(chunk_embeddings)

In [120]:
# --- Question Answering ---

stopwords_id = list(stopwords.words("indonesian"))

# Normalisasi stopwords agar cocok dengan tokenizer Scikit-learn
vectorizer_temp = TfidfVectorizer()
tokenizer = vectorizer_temp.build_tokenizer()
stopwords_id = [" ".join(tokenizer(word)) for word in stopwords_id]  # Pastikan konsistensi

def extract_keywords(question, top_n=5):
    """Mengekstrak kata kunci dari pertanyaan menggunakan TF-IDF (Bahasa Indonesia)"""
    vectorizer = TfidfVectorizer(stop_words=stopwords_id)  # Gunakan stopwords ID yang sudah dinormalisasi
    tfidf_matrix = vectorizer.fit_transform([question])

    feature_array = np.array(vectorizer.get_feature_names_out())
    tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]

    top_keywords = feature_array[tfidf_sorting][:top_n]
    return set(top_keywords)

def filter_chunks_by_keywords(question, chunks):
    """Memilih hanya chunk yang mengandung kata kunci dari pertanyaan"""
    keywords = extract_keywords(question)
    filtered_chunks = [chunk for chunk in chunks if any(keyword.lower() in chunk.lower() for keyword in keywords)]
    return filtered_chunks if filtered_chunks else chunks  # Jika kosong, tetap gunakan semua chunk

def answer_question(question, chunks, top_n=3):
    # Filter chunk berdasarkan kata kunci pertanyaan
    filtered_chunks = filter_chunks_by_keywords(question, chunks)

    # Lakukan embedding hanya pada chunk yang relevan
    filtered_embeddings = embedder.encode(filtered_chunks, convert_to_numpy=True)

    # Buat indeks FAISS baru untuk chunk yang sudah difilter
    index_filtered = faiss.IndexFlatL2(d)
    index_filtered.add(filtered_embeddings)

    # Cari similarity dengan FAISS
    question_embedding = embedder.encode([question], convert_to_numpy=True)
    D, I = index_filtered.search(question_embedding, min(top_n * 2, len(filtered_chunks)))

    # Ambil kandidat chunk berdasarkan FAISS
    candidates = [filtered_chunks[i] for i in I[0]]

    # Gunakan Cross-Encoder untuk memilih chunk terbaik
    pairs = [(question, chunk) for chunk in candidates]
    scores = cross_encoder_model.predict(pairs)
    top_indices = np.argsort(scores)[::-1][:top_n]

    context = "\n".join([candidates[i] for i in top_indices])
    return context

def post_process_answer(answer):
    sentences = sent_tokenize(answer)

    # Hapus duplikasi dan urutkan kalimat agar lebih jelas
    unique_sentences = list(dict.fromkeys(sentences))

    # Format sebagai bullet list dengan memastikan keterbacaan
    bulleted_list = "\n".join([f"* {sentence.strip()}" for sentence in unique_sentences if len(sentence.strip()) > 10])

    return bulleted_list

# TESTING
Chatbot menampilkan mengambil dan menampilkan 3 chunk teks yang paling mirip dengan pertanyaan pengguna sebagai jawaban.

In [121]:
# ===============================
# 7. TESTING CHATBOT
# ===============================

# --- Example Usage ---
question = "Apakah Dana BOSP dapat digunakan untuk pengembangan sumber daya manusia?"  # More focused question
raw_answer = answer_question(question, all_chunks, top_n=3)
processed_answer = post_process_answer(raw_answer)

print(f"Pertanyaan: {question}")
print(f"Jawaban:\n{processed_answer}")

# --- Example Usage ---
question = "Untuk apa saja Dana BOS Kinerja dapat digunakan?"  # More focused question
raw_answer = answer_question(question, all_chunks, top_n=3)
processed_answer = post_process_answer(raw_answer)

print(f"Pertanyaan: {question}")
print(f"Jawaban:\n{processed_answer}")

# --- Example Usage ---
question = "Kapan laporan realisasi penggunaan Dana BOSP harus disampaikan?"  # More focused question
raw_answer = answer_question(question, all_chunks, top_n=3)
processed_answer = post_process_answer(raw_answer)

print(f"Pertanyaan: {question}")
print(f"Jawaban:\n{processed_answer}")

Pertanyaan: Apakah Dana BOSP dapat digunakan untuk pengembangan sumber daya manusia?
Jawaban:
* (2) Komponen penggunaan Dana BOS Kinerja bagi sekolah yang melaksanakan Program Sekolah Penggerak sebagaimana dimaksud pada ayat (1) huruf a meliputi: a. pengembangan sumber daya manusia; b. pembelajaran kurikulum merdeka; c. digitalisasi sekolah; dan d. perencanaan berbasis data.
* (2) Penyampaian laporan realisasi penggunaan Dana BOSP sebagaimana dimaksud pada ayat (1) dilaksanakan paling lambat: a. tanggal Juli tahun anggaran berkenaan untuk laporan realisasi pengunaan Dana BOP PAUD Reguler, Dana BOS Reguler, atau Dana BOP Kesetaraan Reguler tahap I yang ada di Satuan Pendidikan; dan b. tanggal Januari tahun anggaran berikutnya untuk laporan realisasi keseluruhan penggunaan Dana BOSP yang diterima dalam satu tahun anggaran.
* Ketentuan diubah, sehingga berbunyi sebagai berikut: Komponen penggunaan Dana BOP PAUD Kinerja sebagaimana dimaksud dalam ayat (2) huruf b meliputi: a. pengembangan 